In [1]:
## DO NOT RUN THIS.  It is used to build an updated BkAvgAst... table which is already in the processing zip file.
## first step in creating new BkAvgAst... file
## 2017/04/07:  updated for Mar 2017 failure data (Note: only failures before 2017 used since Call data as of Dec-2016)
## 2018/03/08: updated for Dec 2017 failure events (since Call Data only up to Dec17)
## 2018/08/27: updated failure rate details (number fails/number comm bks)
import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_201812.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [3]:
df3 = df2.sort_index(by=['IDRSSD','AsOf'], ascending=True).reset_index(drop=True)
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0           0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
1        8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
2       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
3       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
4       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

   eTotAst   eTotLn   eALLL  eOREO  ...  ncoFIfor  ncoAg  ncoCIdom  ncoCIfor  \
0  65486.0  40507.0   979.0  200.0  ...       0.0    0.0       0.0       0.0   
1  65427.0  42153.0  1049.0    0.0  ...       0.0    0.0       0.0       0.0   
2  65575.0  44359.0  1070.0   40.0  ...       0.0    0.0       0.0       0.0   
3  66624.0  41514.0   987.0   20.0  ...       0.0    0.0       5.0       0.0   
4  68766.0  41539.0  1048.0   20.0  ...       0.0    0.0       0.0       0.0   

   ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0      0.0      0.0        -1.0         0.0       0.0      0.0  
1      0.0      0.0       -23.0         0.0       0.0      0.0  
2      0.0      0.0         2.0         0.0       0.0      0.0  
3      0.0      0.0       104.0         0.0       0.0      0.0  
4      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 196 columns]

In [4]:
len(df3) # 481187 (201612), 504444 (201712), 515693 (201806), 526677 (201812)

526677

In [5]:
'eDDep' in df3.columns

True

In [6]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [7]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [8]:
df6n.tail()

IDRSSD       AsOf
10770  5205819 2018-06-30
10771  5227101 2018-06-30
10772  5278251 2018-09-30
10773  5303724 2018-12-31
10774  5336928 2018-12-31

In [9]:
df4.tail()

IDRSSD       AsOf                         Name State     Cert  \
526672  5227101 2018-12-31             COMMERCEONE BANK    AL  59114.0   
526673  5278251 2018-09-30    CHARLES SCHWAB TRUST BANK    NV  59108.0   
526674  5278251 2018-12-31    CHARLES SCHWAB TRUST BANK    NV  59108.0   
526675  5303724 2018-12-31                GULFSIDE BANK    FL  59140.0   
526676  5336928 2018-12-31  GENERATIONS COMMERCIAL BANK    NY  59149.0   

         eTotAst   aTotAst   eTotLn    eDDep  eFDep  eTotDep  
526672  141634.0  136555.0  63686.0  94512.0    0.0  94512.0  
526673  389012.0  384259.0      0.0   1005.0    0.0   1005.0  
526674  390022.0  390442.0      0.0   1011.0    0.0   1011.0  
526675   35505.0   15070.0   1852.0  12896.0    0.0  12896.0  
526676    2500.0     842.0      0.0      0.0    0.0      0.0

In [10]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2018-12-31            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2018-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2018-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2018-12-31                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [11]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf                        xName xState    xCert  \
10770  5205819 2018-12-31                  STUDIO BANK     TN  59112.0   
10771  5227101 2018-12-31             COMMERCEONE BANK     AL  59114.0   
10772  5278251 2018-12-31    CHARLES SCHWAB TRUST BANK     NV  59108.0   
10773  5303724 2018-12-31                GULFSIDE BANK     FL  59140.0   
10774  5336928 2018-12-31  GENERATIONS COMMERCIAL BANK     NY  59149.0   

           nAsOf  
10770 2018-06-30  
10771 2018-06-30  
10772 2018-09-30  
10773 2018-12-31  
10774 2018-12-31

In [12]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [13]:
df8_orig = df8.copy()
len(df8) # 10762 (201612), 10767 (201712), 10772 (201806), 10775 (201812)

10775

In [14]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2018-12-31            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2018-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2018-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2018-12-31                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   80878.500000   80654.208333   39277.611111   64636.750000   
1 2001-03-31   34006.055556   34342.333333   21659.305556   25982.652778   
2 2001-03-31  154683.583333  149373.652778   77166.791667  107642.763889   
3 2001-03-31    8897.763889    9021.208333    4032.944444    7737.416667   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64636.750000  
1    0.0   25982.652778  
2    0.0  107642.763889  
3    0.0    7737.416667  
4    0.0  144540.423077

In [15]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf                        xName xState    xCert  \
10770  5205819 2018-12-31                  STUDIO BANK     TN  59112.0   
10771  5227101 2018-12-31             COMMERCEONE BANK     AL  59114.0   
10772  5278251 2018-12-31    CHARLES SCHWAB TRUST BANK     NV  59108.0   
10773  5303724 2018-12-31                GULFSIDE BANK     FL  59140.0   
10774  5336928 2018-12-31  GENERATIONS COMMERCIAL BANK     NY  59149.0   

           nAsOf       aeTotAst       aaTotAst       aeTotLn        aeDDep  \
10770 2018-06-30   78099.666667   60883.333333  17858.666667  34512.333333   
10771 2018-06-30  107722.000000   93997.333333  31800.333333  60131.333333   
10772 2018-09-30  389517.000000  387350.500000      0.000000   1008.000000   
10773 2018-12-31   35505.000000   15070.000000   1852.000000  12896.000000   
10774 2018-12-31    2500.000000     842.000000      0.000000      0.000000   

       aeFDep      aeTotDep  
10770     0.0  34512.333333  
10771     0.0  60131.333333  
10772     0.0   1008.000000  
10773     0.0  12896.000000  
10774     0.0      0.000000

In [16]:
ff = pd.read_csv("CBfails_det201812.csv")
ff2 = ff[['State', 'Year', 'Institution Name ','Cert ', 'Effective Date ']]
ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
ff2 = ff2[ff2['FailDate']<'2018-12-31'] #  ONLY USE FAILDATES before Dec 2018!!!!!!!!!!!! (to align with Call Data)
ff2.dropna(subset=['Cert'],inplace=True)
ff2['Cert'] = ff2['Cert'].astype(float)
ff2.head()

State    Year                                             Name     Cert  \
0    IL  2017.0              WASHINGTON FEDERAL BANK FOR SAVINGS  30570.0   
1    KS  2017.0  THE FARMERS AND MERCHANTS STATE BANK OF ARGONIA  17719.0   
2    IL  2017.0                              FAYETTE COUNTY BANK   1802.0   
3    WI  2017.0                                    GUARANTY BANK  30003.0   
4    LA  2017.0                                   FIRST NBC BANK  58302.0   

      EffDate   FailDate  
0  12/15/2017 2017-12-15  
1  10/13/2017 2017-10-13  
2   5/26/2017 2017-05-26  
3    5/5/2017 2017-05-05  
4   4/28/2017 2017-04-28

In [17]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)  # 44

44

In [18]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs (28, now 44) (assistance, then failure)

44

In [19]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_index(by=['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State    Year                              Name    Cert     EffDate  \
2374    TX  1988.0        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
705     TX  1992.0    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
2375    TX  1988.0      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
706     TX  1992.0  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
2373    TX  1988.0         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
2374 1988-04-20  
705  1992-10-30  
2375 1988-04-20  
706  1992-10-30  
2373 1988-04-20

In [20]:
## sections to dedup Cert field: (saves latest date, which usually is the fail date vs assistance date)

In [21]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
3559  90257.0 1992-08-21  x
3560  90266.0 1992-09-18  x
3561  90307.0 1992-08-14  x
3562  90310.0 1991-06-07  x
3563  91189.0 1998-07-23  x

In [22]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State    Year                             Name     Cert    EffDate  \
3603    MO  1970.0  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
3604    NJ  1970.0          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
3605    KY  1970.0       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
3606    MI  1970.0   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
3607    IA  1970.0       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
3603 1970-08-24  x  
3604 1970-08-07  x  
3605 1970-06-25  x  
3606 1970-04-18  x  
3607 1970-02-22  x

In [23]:
ff4=ff3.dropna()
len(ff3), len(ff4) # 3608, 3564

(3608, 3564)

In [24]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8) # 10775

10775

In [25]:
df8.sort_index(by='aeTotAst',ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.482006e+09  1.456936e+09  5.203546e+08  6.834806e+08   
1 2001-03-31  1.270022e+09  1.273816e+09  6.647306e+08  8.105992e+08   
2 2001-03-31  1.063243e+09  1.060575e+09  5.082855e+08  2.856345e+08   
3 2001-03-31  8.926864e+08  8.810598e+08  5.379178e+08  6.205972e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.541728e+08  9.376534e+08        NaT  
1  1.054239e+08  9.160231e+08 2009-01-16  
2  4.414540e+08  7.270885e+08 2008-11-23  
3  6.737132e+07  6.879685e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [26]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.482006e+09  1.456936e+09  5.203546e+08  6.834806e+08   
1 2001-03-31  1.270022e+09  1.273816e+09  6.647306e+08  8.105992e+08   
2 2001-03-31  1.063243e+09  1.060575e+09  5.082855e+08  2.856345e+08   
3 2001-03-31  8.926864e+08  8.810598e+08  5.379178e+08  6.205972e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.541728e+08  9.376534e+08        NaT  9.202367   9.202367  5.670701   
1  1.054239e+08  9.160231e+08 2009-01-16  7.886074  17.088441  7.244076   
2  4.414540e+08  7.270885e+08 2008-11-23  6.602099  23.690540  5.539174   
3  6.737132e+07  6.879685e+08        NaT  5.543047  29.233587  5.862100   
4  2.469325e+07  2.951088e+08        NaT  2.742359  31.975946  2.774973   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.670701  7.448409   7.448409  
1  12.914777  8.833718  16.282127  
2  18.453951  3.112777  19.394904  
3  24.316051  6.763120  26.158025  
4  27.091024  2.946925  29.104950

In [27]:
df8.to_csv("BkAvgAstPR_FailDate_201812.csv")

In [28]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail) # 513

513

In [29]:
df8_fail.to_csv("FailedBkList_AvgAst_201812.csv")